In [1]:
import os
import pandas as pd

In [2]:
data = pd.read_csv(r'C:\Users\carswje\OneDrive - HII\Documents\Fleet-Sustainment-HVAC-Project\RawData\cvn-hvac-jobs-data.csv') #47350 x 87

C:\Users\carswje\AppData\Local\Temp\ipykernel_29236\3477807849.py:1: DtypeWarning: Columns (3,4,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r'C:\Users\carswje\OneDrive - HII\Documents\Fleet-Sustainment-HVAC-Project\RawData\cvn-hvac-jobs-data.csv') #47350 x 87


# Removing Unnecessary Columns

In [3]:
#Removing columns providing no value. Dimension is now 47350 x 56
data = data.drop(['alt_closing',
'alt_opening',
'insurv_number',
'identification_number_closing',
'insurv_suffix',
'eswbs_closing',
'integrated_priority',
'meter_reading_opening',
'special_tycom_data',
'mcc',
'remarks',
'UsedinMerit',
'OARSCreatedDate',
'OARSUpdateDate',
'CASREP_CAT',
'CASREP',
'idCause',
'idSymptom',
'idSystem',
'idSubsystem',
'idEquipmentgroup',
'idEquipment',
'idComponent',
'idSubcomponent',
'idAssess',
'completed',
'rpr_lvl',
'ref_num',
'foundinMerit',
'configurationItemId',
'ship_readiness_effect',
'closing_ship_system',
'assigned',
'casrep_filed',
'total_material_cost'], axis=1) 

# Handling NAs

In [4]:
na_counts = data.isnull().sum()

print(f"{'Column Name':<40} | {'# of NAs'}")
print("-" * 40 + " | " + "-" * 8)

for col, count in na_counts.items():
    print(f"{col:<40} | {count}")

Column Name                              | # of NAs
---------------------------------------- | --------
uic                                      | 0
job_seq                                  | 0
action_taken_code                        | 11233
apl                                      | 86
cause_code                               | 0
cdm_rin                                  | 948
csmp_narrative_summary                   | 14
date_closing                             | 11233
date_maintenance_action                  | 0
deferral_reason_code                     | 24
due_date                                 | 27220
eic                                      | 10
equipment_nomenclature                   | 1
eswbs_opening                            | 0
iuc_screening_code                       | 20663
jcn                                      | 0
location                                 | 176
mhc                                      | 0
mho                                      | 0
mhr              

In [5]:
#Action_taken and action_taken_code NA Handling

mask = data['action_taken_code'].isna() & data['action_taken'].isna() #When both are NA (occurs 11233 times), the job is still Open
data.loc[mask, 'action_taken_code'] = 'Open_job'
data.loc[mask, 'action_taken'] = 'Open_job'

#After replacing the 11223 NAs that are associated with open jobs with 'Open_job, there are 597 remaining NAs. 
#6I and 3B are the input for 474 of the 597 and they do not have a corresponding text column. 
#Create a new category 'Unknown' for these 597.
data['action_taken'] = data['action_taken'].fillna('Unknown')

In [6]:
#apl NA Handling 
#86 NAs, replacing with mode
data['apl'] = data['apl'].fillna('XSYSTEMITEM')

In [7]:
#cdm_rin NA Handling 
#948 NAs
#Creating new placeholder value '-----' to replace NAs
data['cdm_rin'] = data['cdm_rin'].fillna('-----')

In [8]:
#csmp_narrative_summary NA Handling
#14 NAs
data['csmp_narrative_summary'] = data['csmp_narrative_summary'].fillna('Unknown')

In [9]:
#date_closing NA Handling
#11223 NAs, represent Open Jobs
#errors = 'coerce' replaces Missing Values with NaT (Like NaN, but specifically for dates. Allows for date calculations)
data['date_closing'] = data['date_closing'] = pd.to_datetime(data['date_closing'], errors = 'coerce')

In [10]:
#deferral_reason and deferral_reason_code NA Handling
#Both have 13 NAs
#35% are code 2: 'Lack of material'
#Replace NA with mode

data['deferral_reason_code'] = data['deferral_reason_code'].fillna(2.0)
data['deferral_reason'] = data['deferral_reason'].fillna('Lack of material')

In [11]:
#due_date NA Handling
#27220 NAs 
data['due_date'] = pd.to_datetime(data['due_date'], errors = 'coerce')

In [12]:
#eic NA Handling 
#10 NAs. Replacing with mode, seen 57% of the time
data['eic'] = data['eic'].fillna('T200000')

In [13]:
#equipment_nomenclature NA Handling
#1 NA. Filling with mode, seen ~ 50% of the time
data['equipment_nomenclature'] = data['equipment_nomenclature'].fillna('VENT NONMCH PRP')

In [14]:
#iuc_screening_code and intermediate_unit_commander_screening NA Handling

data['iuc_screening_code'] = data['iuc_screening_code'].fillna(0)
data['intermediate_unit_commander_screening'] = data['intermediate_unit_commander_screening'].fillna('Unknown')

In [15]:
#Location NA Handling 
#176 NAs
#Trying to maintain format of location records
data['location'] = data['location'].fillna('?-?-?-?')

In [16]:
#mhr NA Handling
#592 NAs. Most records (>80%) have 0
data['mhr'] = data['mhr'].fillna(0)

In [17]:
#priority_code NA Handling
#25 NAs. ~ 98% have code 4, replacing with mode
data['priority_code'] = data['priority_code'].fillna(4)

In [18]:
#safety_code NA Handling
#42702 NAs
#Convert to binary variable. NA represents No safety hazard (0) and non-NA are treated as safety hazard (1)
data['safety_code'] = data['safety_code'].notna().astype(int)

In [19]:
#status_code NA Handling
#1 NA, replace with mode
data['status_code'] = data['status_code'].fillna(1)

In [20]:
#tycom_screening_code (9026 NA) and tycom_screening (9031) NA handling
#Create new category 0
data['tycom_screening_code'] = data['tycom_screening_code'].fillna(0)

data['tycom_screening'] = data['tycom_screening'].fillna('Unknown')


In [21]:
#type_availability_code NA Handling
#25 NAs. Replaced with mode
data['type_availability_code'] = data['type_availability_code'].fillna(4)

In [22]:
#total_ima_man_hours, total_ship_force_man_hours, total_replacement_cost, total_repair_replacement_cost NA Handling
#All have 79 NAs. Replace with median
data['total_ima_man_hours'] = data['total_ima_man_hours'].fillna(data['total_ima_man_hours'].median())
data['total_ship_force_man_hours'] = data['total_ship_force_man_hours'].fillna(data['total_ship_force_man_hours'].median())
data['total_replacement_cost'] = data['total_replacement_cost'].fillna(data['total_replacement_cost'].median())
data['total_repair_replacement_cost'] = data['total_repair_replacement_cost'].fillna(data['total_repair_replacement_cost'].median())



In [23]:
#Narrative Column NA Handling
#Replacing NAs with 'Unknown', free form text columnns
#def_narrative 13501 NA
#nondef_narrative 22501 NA
#closing_narrative 26364 NA
#ima_narrative 46059 NA
data['def_narrative'] = data['def_narrative'].fillna('Unknown')
data['nondef_narrative'] = data['nondef_narrative'].fillna('Unknown')
data['closing_narrative'] = data['closing_narrative'].fillna('Unknown')
data['ima_narrative'] = data['ima_narrative'].fillna('Unknown')

In [25]:
#priority NA Handling 
# Note to self - priority_code is also in the dataset 
#25 NAs here too. Replace with mode
data['priority'] = data['priority'].fillna('Desirable')

In [26]:
#component status NA Handling 
#1 NA. Replace with mode
data['component_status'] = data['component_status'].fillna('Operational')

In [27]:
#type_of_availability_needed NA Handling
#25 NAs
#Replace with mode
data['type_of_availability_needed'] = data['type_of_availability_needed'].fillna("Ship's force")

In [28]:
#when_discovered NA Handling 
#2637 NAs 
#Replace with no failure, pms accomplishment only.
#  Every single NA for when_discovered has 9 as the value for when_discovered_code. 9: no failure, pms accomplishment only
data['when_discovered'] = data['when_discovered'].fillna('no failure, pms accomplishment only')

In [29]:
#Now all NAs handled except 
#date_closing and due_date: treated as NaT

#Next,
#Ensure correct data types [Here] 
#Account for irregular inputs ex: negative days_open value
#Outlier vs. misinput, normalizating, etc. for numeric columns


# Assigning Appropriate Data Types

In [ ]:
data.dtypes

uic                                               object
job_seq                                            int64
action_taken_code                                 object
apl                                               object
cause_code                                         int64
cdm_rin                                           object
csmp_narrative_summary                            object
date_closing                              datetime64[ns]
date_maintenance_action                           object
deferral_reason_code                             float64
due_date                                  datetime64[ns]
eic                                               object
equipment_nomenclature                            object
eswbs_opening                                     object
iuc_screening_code                                object
jcn                                               object
location                                          object
mhc                            

In [43]:
for col in data.columns:
    name = f"{col:<{45}}"
    dtype =f"{str(data[col].dtype):<{30}}"
    entry = f"{'enter here':<{40}}"
    print(f"{name}{dtype}{entry}")

uic                                          object                        enter here                              
job_seq                                      int64                         enter here                              
action_taken_code                            object                        enter here                              
apl                                          object                        enter here                              
cause_code                                   int64                         enter here                              
cdm_rin                                      object                        enter here                              
csmp_narrative_summary                       object                        enter here                              
date_closing                                 datetime64[ns]                enter here                              
date_maintenance_action                      object                     

# Columns, Their Current Data Types, and Their Intended Data Types



uic                                          object                        enter here                              
job_seq                                      int64                         enter here                              
action_taken_code                            object                        enter here                              
apl                                          object                        enter here                              
cause_code                                   int64                         enter here                              
cdm_rin                                      object                        enter here                              
csmp_narrative_summary                       object                        enter here                              
date_closing                                 datetime64[ns]                enter here                              
date_maintenance_action                      object                        enter here                              
deferral_reason_code                         float64                       enter here                              
due_date                                     datetime64[ns]                enter here                              
eic                                          object                        enter here                              
equipment_nomenclature                       object                        enter here                              
eswbs_opening                                object                        enter here                              
iuc_screening_code                           object                        enter here                              
jcn                                          object                        enter here                              
location                                     object                        enter here                              
mhc                                          int64                         enter here                              
mho                                          int64                         enter here                              
mhr                                          float64                       enter here                              
priority_code                                float64                       enter here                              
safety_code                                  int32                         enter here                              
status_code                                  float64                       enter here                              
tycom_screening_code                         object                        enter here                              
type_availability_code                       float64                       enter here                              
type_of_maintenance_action                   object                        enter here                              
when_discovered_code                         int64                         enter here                              
work_center                                  object                        enter here                              
total_ima_man_hours                          float64                       enter here                              
total_ship_force_man_hours                   float64                       enter here                              
total_replacement_cost                       float64                       enter here                              
total_repair_replacement_cost                float64                       enter here                              
def_narrative                                object                        enter here                              
nondef_narrative                             object                        enter here                              
closing_narrative                            object                        enter here                              
ima_narrative                                object                        enter here                              
opening_ship_system                          object                        enter here                              
job_status                                   object                        enter here                              
days_open                                    int64                         enter here                              
action_taken                                 object                        enter here                              
intermediate_unit_commander_screening        object                        enter here                              
priority                                     object                        enter here                              
component_status                             object                        enter here                              
type_of_availability_needed                  object                        enter here                              
tycom_screening                              object                        enter here                              
when_discovered                              object                        enter here                              
cause                                        object                        enter here                              
deferral_reason                              object                        enter here                              
feasible_with_ship_inventory                 bool                          enter here                              
feasible_with_additional_dla_inventory       bool                          enter here                              
ship                                         object                        enter here                              
shipname                                     object                        enter here                               